In [1]:
import open3d as o3d
import numpy as np
import math
import copy

In [2]:
def load_mesh_model(path):
    """
    Description
    -----------
        read mesh model

    Parameters
    ----------
        path: str

    Returns
    -------
        o3d.geometry.TriangleMesh
    """
    mesh = o3d.io.read_triangle_mesh(path)
    return mesh

def sample_mesh_to_cloud(mesh, sample_number=100000):
    """
    Description
    -----------
        process the cabinet mesh model, sample it as point cloud

    Parameters
    ----------
        mesh: o3d.geometry.TriangleMesh
        sample_number: int

    Returns
    -------
        o3d.geometry.PointCloud
    """
    mesh.compute_vertex_normals()
    pcd = mesh.sample_points_uniformly(number_of_points=sample_number)
    pcd.paint_uniform_color([1, 0.706, 0])  # yellow
    return pcd

In [7]:
from pyquaternion import Quaternion


def matrix2rotation(matrix):
    return matrix[:3, :3]


def matrix2qua(matrix):
    qua = Quaternion(matrix=matrix2rotation(matrix))
    return qua


def matrix2trans(matrix):
    return np.array(matrix[:3, 3:].transpose())[0]


def matrix2pose(matrix):
    qua = matrix2qua(matrix)
    translation = matrix2trans(matrix)
    return np.hstack((translation, qua.elements))


def quatrans2matrix(qua, trans):
    rotaion_matrix = qua.rotation_matrix
    trans_matrix = np.array([trans]).transpose()
    tf_m = np.hstack((rotaion_matrix, trans_matrix))
    m = np.mat([[0, 0, 0, 1]])
    tf_m = np.vstack((tf_m, m))

    return tf_m


def pose2matrix(pose):
    w, x, y, z = pose[3:]
    qua = Quaternion(w=w, x=x, y=y, z=z)
    trans = pose[:3]
    return quatrans2matrix(qua, trans)


def translation2matrix(trans):
    qua = Quaternion(w=1, x=0, y=0, z=0)
    return quatrans2matrix(qua, trans)


def qua2matrix(qua):
    trans = [0, 0, 0]
    return quatrans2matrix(qua, trans)

In [11]:
path = "/home/bot/dev/projects_data/kandovan/cur/dd/model_center_m.STL"
mesh = load_mesh_model(path)
target = sample_mesh_to_cloud(mesh, 500000)
# target.scale(0.001, center=False)
target.transform(pose2matrix([0,0,-0.075,0.707,0,0,0.707]).A)


world_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(1)
o3d.visualization.draw_geometries([world_frame, target])

In [4]:
def load_point_cloud(path):
    """
    Description
    -----------
        read point cloud

    Parameters
    ----------
        path: str

    Returns
    -------
        o3d.geometry.PointCloud
    """
    cloud = o3d.io.read_point_cloud(path)
    return cloud

In [9]:
source = load_point_cloud("/home/bot/dev/projects_data/kandovan/cur/tfed_model.pcd")
o3d.visualization.draw_geometries([world_frame, source])

In [12]:
o3d.visualization.draw_geometries([world_frame, source, target])

In [14]:
output_path = "/home/bot/dev/projects_data/kandovan/cur/dd/model_center_m.pcd"
o3d.io.write_point_cloud(output_path, target)

True